# Homework 3 - What is the best anime in the world?

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import os
import pickle
import numpy as np
import time
import re
import datetime as dt 
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import heapq
from utilities import *

C:\Users\aurel\Downloads\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\aurel\Downloads\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guida

IndentationError: expected an indented block (utilities.py, line 21)

## 1. Data collection

### 1.1. Get the list of animes

First, we obtain the urls for the top 20000 animes listed in the top anime series tier list. This is done across approximately 400 pages. Once we've collected all the urls, we save them in a txt file.

In [3]:
if "urls.txt" not in os.listdir():
    urls = take_n_urls(20000)
    # Since the output of this step has to be a txt file, here we write one with each
    # url separated by a newline
    with open("urls.txt", "w") as file:
        file.write("\n".join(urls_str))
else:
    with open("urls.txt", "r", encoding="utf8") as file:
        print("Loading urls...")
        urls = file.read().split("\n")
        print("Done!")

Loading urls...
Done!


We end up with 19131 urls.

In [4]:
# We check the lenght of our list of URL which is actually 19103 instead of 20000 as it is said in the homework. 

print(len(urls)) 

19130


In [5]:
urls_str = list(map(str, urls))

### 1.2. Crawl animes

We then proceeded to save the htmls for each of this urls.
After approximately 175 requests the website blocks any further requests so we added some checks to the status_code of each request in order to wait some time if we had been locked out. We also designed a system than allowed for multiple members of our team to automatically continue the collection of html pages from where it was left off at the last git push.

##### Further details on our strategy for these issues are in the documentation of the function (```utilities.py``` file).

In [4]:
# Here we create the directory where the html pages will be stored
if "html_pages" not in os.listdir():
    os.mkdir("html_pages")

In [5]:
save_html_pages(urls)

Starting from anime #19130


### 1.3 Parse downloaded pages

Then we proceeded to create tsv files for each anime with the fields recommended in the homework description.

1. Anime Name (to save as `animeTitle`): String
2. Anime Type (to save as `animeType`): String
3. Number of episode (to save as `animeNumEpisode`): Integer
4. Release and End Dates of anime (to save as `releaseDate` and `endDate`): Convert both release and end date into datetime format.
5. Number of members (to save as `animeNumMembers`): Integer
6. Score (to save as `animeScore`): Float
7. Users (to save as `animeUsers`): Integer
8. Rank (to save as `animeRank`): Integer
9. Popularity (to save as `animePopularity`): Integer
10. Synopsis (to save as `animeDescription`): String
11. Related Anime (to save as `animeRelated`): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. List of strings.
12. Characters (to save as `animeCharacters`): List of strings.
13. Voices (to save as `animeVoices`): List of strings
14. Staff (to save as `animeStaff`): Include the staff name and their responsibility/task in a list of lists.


In [6]:
# Here we create the directory where the tsv files will be stored
if "tsv_files" not in os.listdir():
    os.mkdir("tsv_files")
    for i in range(len(urls)):
        collect_info(i) #you can find details in utilities.py 

## 2. Search Engine

### 2.0. Pre-processing

1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed

Our next step consisted in merging all the tsvs in a single dataframe and then preprocessing the synopsis for each anime. This preprocessing consisted in tokenizing the words with nltk, removing punctuation and stopwords and then stemming the results. The stemming was achieved specifically through the Porter stemmer, but the function is also able to perform the same process with the Lancaster stemmer.
<br>
The dataframe with the preprocessed synopsis is saved as a binary file for further use, while the original dataframe is saved as a regular csv.

In [8]:
path = "./tsv_files/"
colnames = ["animeTitle", "animeType", "animeNumEpisode", "releaseDate", "endDate", "animeNumMembers",
            "animeScore", "animeUsers", "animeRank", "animePopularity", "animeDescription", "animeRelated",
            "animeCharacters", "animeVoices", "animeStaff"]

if "html_df.csv" not in os.listdir():
    df = merge_tsvs(path, colnames)
    # Save our df in csv format
    df.to_csv("./html_df.csv")
else:
    df = pd.read_csv("html_df.csv")

In [9]:
# create (or load if it already exists) a copy of our dataframe with an extra column containing the preprocessed synopsis
if "df_with_tokens.p" not in os.listdir():
    df_new = df.assign(tokenized_desc=df["animeDescription"].apply(lambda x: process_text(x)))
    with open("df_with_tokens.p", "wb") as file:
        pickle.dump(df_new, file)
else:
    with open("df_with_tokens.p", "rb") as file:
        df_new = pickle.load(file)

In [10]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19130 entries, 0 to 19129
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   animeTitle        19130 non-null  object 
 1   animeType         19130 non-null  object 
 2   animeNumEpisode   18620 non-null  float64
 3   releaseDate       18760 non-null  object 
 4   endDate           8451 non-null   object 
 5   animeNumMembers   19130 non-null  int64  
 6   animeScore        13436 non-null  float64
 7   animeUsers        13436 non-null  float64
 8   animeRank         17307 non-null  float64
 9   animePopularity   19130 non-null  int64  
 10  animeDescription  19130 non-null  object 
 11  animeRelated      12706 non-null  object 
 12  animeCharacters   19130 non-null  object 
 13  animeVoices       19130 non-null  object 
 14  animeStaff        10247 non-null  object 
 15  tokenized_desc    19130 non-null  object 
dtypes: float64(4), int64(2), object(10)
memo

### 2.1. Conjunctive query
#### 2.1.1) Create your index!

Now we create a vocabulary file that maps each word to an integer ("term_id").
We also create an inverted index that for each term_id maps to the documents that contain that word, according to the following format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```

where _document\_i_ is the *id* of a document that contains the word.


##### You can find the documentation of those functions in ```utilities.py```

In [11]:
#define the names of the interesting files 
name_voc_file = "vocabulary.pkl"
name_file_corpus = 'df_with_tokens.p'
name_inv_ind_file = "inverted_index.pkl"

#download the corpus and the vocabulary
corpus = download_corpus(name_file_corpus)
voc = download_voc(corpus, name_voc_file)
inv_ind = download_inverted_index(corpus,voc, name_inv_ind_file)

#### 2.1.2) Execute the query

Next we created a search engine that, given a query, returns the documents that contain all the words in the query.
<br><br>
Specifically, for each documents, as per homework specifications, we return:

* `animeTitle`
* `animeDescription`
* `Url`

Here the query is get with an ```input()``` method. We test our search engine on the query "saiyan race":

##### You can find the documentation of this function in ```utilities.py```

In [22]:
search_engine_1(voc, inv_ind, urls)

What is your query ?saiyan race


,animeTitle,animeDescription,Url
364,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z
401,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
1035,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...
1467,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...
1961,Dragon Ball Super,"Seven years after the events of Dragon Ball Z,...",https://myanimelist.net/anime/30694/Dragon_Bal...
2015,Dragon Ball Z Movie 14: Kami to Kami,"Following the defeat of a great adversary, Gok...",https://myanimelist.net/anime/14837/Dragon_Bal...
2296,"Dragon Ball Z Movie 08: Moetsukiro!! Nessen, R...",As Goku investigates the destruction of the So...,https://myanimelist.net/anime/901/Dragon_Ball_...
4339,Dragon Ball: Ossu! Kaettekita Son Gokuu to Nak...,Based on an original concept by the original a...,https://myanimelist.net/anime/5152/Dragon_Ball...
4673,Dragon Ball Z Movie 10: Kiken na Futari! Super...,"After his loss to Goku, Broly crash lands and ...",https://myanimelist.net/anime/903/Dragon_Ball_...
5664,Dragon Ball Z: Summer Vacation Special,"One peaceful afternoon, the Son family and fri...",https://myanimelist.net/anime/22695/Dragon_Bal...


### 2.2) Conjunctive query & Ranking score
#### 2.2.1) Inverted index

We created several functions that helped us achieve the goal of this exercise. 
<br><br>
First, we created a function that calculates the tfidf score for each word and document provided. This function returns not only the tfidf, but also, to achieve slightly faster computational times, the idf scores since they do not depend on the specific document at hand and can be computed just once for each word.
<br><br>
We also created another inverted index that stores for each term_id not only the documents that include that word, but also their respective tfidf scores.
<br><br>
Finally, using this inverted index, our search engine retrieves the documents that satisfy all the words in a query, computes the cosine distance between their tfidf and the one from the query (which is treated as its own document), reorganize them in a max-heap structure according to the cosine similarity score, and returns the top k results (if less than k results are available for that specific query, it returns all the results it finds).

For each output we display the fields animeTitle, animeDescription, Url and similarity score.

##### You can find the documentation of those functions in ```utilities.py```

In [13]:
ii_2, idfs = download_inverted_index_tfidf(corpus,voc, "inverted_index_2.p", "idfs.p")

Here we first test it with the query "sayian goku vegeta": (k=10)

In [28]:
query = "sayian goku vegeta"
output = search_k_matches(query, corpus, voc, ii_2, idfs, urls)
output

,animeTitle,animeDescription,Url,Similarity
4798,Dragon Ball Z: Saiya-jin Zetsumetsu Keikaku,Dr. Raichii is the only Tsufurujin (the race e...,https://myanimelist.net/anime/984/Dragon_Ball_...,0.996483
2935,Dragon Ball: Episode of Bardock,"Shown at Jump Festa 2012, held in December, 20...",https://myanimelist.net/anime/12231/Dragon_Bal...,0.996483
3338,Dragon Ball Z Movie 09: Ginga Girigiri!! Bucch...,Mr. Money is holding another Tenka'ichi Budoka...,https://myanimelist.net/anime/902/Dragon_Ball_...,0.996483
2296,"Dragon Ball Z Movie 08: Moetsukiro!! Nessen, R...",As Goku investigates the destruction of the So...,https://myanimelist.net/anime/901/Dragon_Ball_...,0.971844
8034,Dragon Ball Z: The Real 4-D,Goku and Frieza may have battled in the manga ...,https://myanimelist.net/anime/33947/Dragon_Bal...,0.918850
4413,Dragon Ball Z Movie 07: Kyokugen Battle!! Sand...,"Dr. Gero's Androids #13, #14, and #15 are awak...",https://myanimelist.net/anime/900/Dragon_Ball_...,0.853809


And then we test it against the keywords "ranma urusei" which are two characters from two different animes that only meet in a special crossover episode called "It's a Rumic World". We can see that the search engine correctly gives this result as the best match, followed by another Ranma 1/2 episode. No other results satisfy this query.

In [29]:
query = "ranma urusei"
output = search_k_matches(query, corpus, voc, ii_2, idfs, urls)

In [30]:
output

,animeTitle,animeDescription,Url,Similarity
4286,It's a Rumic World: 50th Anniversary Weekly★Sh...,"The characters from Ranma 1/2, Urusei Yatsura,...",https://myanimelist.net/anime/6566/Its_a_Rumic...,0.997792
1160,Ranma ½ Super,Super OVA 1: Based on a story from vol. 27 of ...,https://myanimelist.net/anime/1011/Ranma_½_Super,0.910875


## 3. Define a new score!


For this exercise we designed an algorithm that allows the user to specify certain parameters (specifically, **year of release**, **voice actors**, **characters**, **related animes** and **type of episode**) in addition to the main query, in order to get better results for the search.
<br><br>
Specifically, the user can submit the query in one of two ways: either directly in the format "query \[parameter1=x, parameter2=y...\]" or, if the string is not provided, through a form that collects the data for each field and then assembles the same string so that the inner logic of the algorithm doesn't change regardless of the way the query is collected.
<br><br>
The function process_query then collects each field and returns a dictionary containing all the relevant information.
<br><br>
We then proceed to evaluate the cosine distance between the tfidf scores of each document including all the words in the query and the query itself as we did before, but we also assess whether the fields specified in the query match with those of the documents we analyzed and we accordingly create a specific vector of booleans called vector_parameters.

This vector is then used to compute the Tanimoto Distance with the query (we assume for the query a vector of all 1s).

The reason why we adopted a specific distance measure for the parameters rather than using the cosine distance is that the latter performs very poorly on boolean vectors.

Finally, we average the cosine distance computed on the tfidf scores and the Tanimoto distance computed on the parameters and obtain a single score, which is then used to again organize the results in a max-heap structure and return the top k results.

It's important to notice that since the Tanimoto distance is computed across five different parameters, each parameter weighs 1/5 of the total score and thus does not have the potential, alone, to overturn a match in the main query.

This concept will be made more clear in the following examples.

##### You can find the documentation of those functions in ```utilities.py```

***First Experiment***
<br><br>
For our first example we use the main query "ranma urusei" which should point more to #4286 (because that episode features both characters), and we provide the correct year and genre. 

Everything else is either wrong or missing. We end up with a similarity of approximately 80%.

In [31]:
# example with query provided
query = "ranma urusei [year=2008, type=special, voices=(gianni), related=(gemelli del destino), characters=(ranma)]"
search_k_matches_2(corpus, voc, ii_2, idfs, urls, query=query)

,animeTitle,animeDescription,Url,Similarity
4286,It's a Rumic World: 50th Anniversary Weekly★Sh...,"The characters from Ranma 1/2, Urusei Yatsura,...",https://myanimelist.net/anime/6566/Its_a_Rumic...,0.798896
1160,Ranma ½ Super,Super OVA 1: Based on a story from vol. 27 of ...,https://myanimelist.net/anime/1011/Ranma_½_Super,0.555438


***Second experiment***

Now we keep the same main query but we put the year for the other episode. 

This should raise the similarity for #1160 and lower the one for #4286. This is exactly what happens, but since the cosine similarity has more weight than the single coefficients for the Tanimoto (so the parameters), the top result is still the one that better matches the main query "ranma urusei".

In [32]:
query = "ranma urusei [year=1995, type=special, voices=(gianni), related=(gemelli del destino), characters=(ranma)]"
search_k_matches_2(corpus, voc, ii_2, idfs, urls, query=query)

,animeTitle,animeDescription,Url,Similarity
4286,It's a Rumic World: 50th Anniversary Weekly★Sh...,"The characters from Ranma 1/2, Urusei Yatsura,...",https://myanimelist.net/anime/6566/Its_a_Rumic...,0.698896
1160,Ranma ½ Super,Super OVA 1: Based on a story from vol. 27 of ...,https://myanimelist.net/anime/1011/Ranma_½_Super,0.655438


***Third experiment***

Now we enter also the anyme type of the other episode. 

At this point, since we have two parameters pointing toward #1160, the order is inverted and we get #1160 as the first result. Again, this makes sense because at this point the query seems to match more #1160 than #4286.

In [33]:
query = "ranma urusei [year=1995, type=ova, voices=(gianni), related=(gemelli del destino), characters=(ranma)]"
search_k_matches_2(corpus, voc, ii_2, idfs, urls, query=query)

,animeTitle,animeDescription,Url,Similarity
1160,Ranma ½ Super,Super OVA 1: Based on a story from vol. 27 of ...,https://myanimelist.net/anime/1011/Ranma_½_Super,0.755438
4286,It's a Rumic World: 50th Anniversary Weekly★Sh...,"The characters from Ranma 1/2, Urusei Yatsura,...",https://myanimelist.net/anime/6566/Its_a_Rumic...,0.598896


***Query retrieval via form***

Here is an example of the query retrieval via form

In [21]:
# example with query via form
search_k_matches_2(corpus, voc, ii_2, idfs, urls)

Enter your query: ranma urusei
Year it was released: 2008
Type of anime: special
Voice actors: gianni
Characters: sandro
Related animes: 


,animeTitle,animeDescription,Url,Similarity
4286,It's a Rumic World: 50th Anniversary Weekly★Sh...,"The characters from Ranma 1/2, Urusei Yatsura,...",https://myanimelist.net/anime/6566/Its_a_Rumic...,0.748896
1160,Ranma ½ Super,Super OVA 1: Based on a story from vol. 27 of ...,https://myanimelist.net/anime/1011/Ranma_½_Super,0.455438


## 5. Algorithmic question

##### Problem :
You consult for a personal trainer who has a *back-to-back sequence* of requests for appointments. A sequence of requests is of the form
    > 30, 40, 25, 50, 30, 20
where each number is the time that the person who makes the appointment wants to spend.
You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, `[30, 50, 20]` is an acceptable solution (of duration *100*), but `[30, 40, 50, 20]` is not, because *30* and *40* are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is `[40, 50, 20]`, of total duration *110*.

### 1. Write an algorithm that computes the acceptable solution with the longest possible duration.

##### Here we consider that all values in the instance are unique and len(instance) = n

To compute the acceptable solution with the longest possible duration, we have to follow several steps :
1. Compute every possible solution : So for that, list all sublists which represent each possible list of appointments.
2. For each sublist, tell if it is acceptable or not, so if there are two consecutive appointments or not.
3. Compute the total duration of each acceptable solution.
4. Finally, return the solution which correspond to the maximum duration.

```
Input: 
    instance: list of length n

function optimal_solution(instance):
    n=len(instance)
    for i=0 to n: 
        sublists = sublists + [all sublists with i elements]
    
    acceptable_solutions=[all element of sublists which are acceptable]
    
    durations = [duration of each element of acceptable_solutions]
    max_duration = max(durations)
    optimal_solutions = [sublists of instance with total duration == max_durations]
    
    return optimal_solutions, max_duration
```

### 2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

First of all, we create a function ```is_acceptable(solution, instance)``` that says if a solution is acceptable or not, i.e. there is not two consecutive requests of the instance in the solution.

Then, the ```longest_acceptable_duration(instance)``` compute all possible sublists of the instance, test it with the function ```is_acceptable(solution, instance)```, sum every acceptable list (to compute the duration of each acceptable solution) and return the maximum. 

In [142]:
def is_acceptable(solution, instance):
    res=True 
    for x,y in zip(solution[:-1],solution[1:]):
        i= instance.index(x)
        #if the next element is y 
        if instance[i+1] == y:
            res = False
    return res

def optimal_solution(instance):
    sublists=[]
    
    for i in range(1, len(instance)+1):
        sublists+=[list(x) for x in combinations(instance, i)]
        
    mask = [is_acceptable(solution, instance) for solution in sublists]
    acceptable_sol = np.array(sublists)[mask]
    
    durations = [sum(L) for L in acceptable_sol]
    max_duration = max(durations)
    
    index_optimal_solutions = np.where(np.array(durations)==max_duration)
    
    return list(np.array(acceptable_sol)[index_optimal_solutions]), max_duration

In [143]:
instance = [30, 40, 25, 50, 30, 20]
optimal_solutions, max_duration = optimal_solution(instance)
print(optimal_solutions, max_duration)

[[40, 50, 20]] 110
